In [1]:
from rdkit import Chem
import pandas as pd

In [2]:
kinase_list = ['ABL1', 'AKT1', 'AKT2', 'BRAF', 'CDK2', 'CSF1R', 'EGFR', 
               'FAK1', 'FGFR1', 'IGF1R', 'JAK2', 'KIT', 'KPCB', 'LCK', 
               'MAPK2', 'MET', 'MK01', 'MK10', 'MK14', 'MP2K1', 'PLK1', 
               'ROCK1', 'SRC', 'TGFR1', 'VGFR2', 'WEE1']

In [19]:
# Function to read an SDF file and return a list of dictionaries with molecule data
def read_sdf_to_dict(sdf_file, target):
    supplier = Chem.SDMolSupplier(sdf_file)
    molecule_data = []

    for mol in supplier:
        if mol is None:
            continue
        
        # Get the SMILES string
        smiles = Chem.MolToSmiles(mol)
        
        # Get the molecule's name (if available)
        mol_name = mol.GetProp("_Name") if mol.HasProp("_Name") else "Unknown"
        
        if mol_name[:4]=="CHEM":
            kind = "actives"
        else:
            kind = "decoys"
        
        # You can extract more properties if needed using mol.GetProp("property_name")
        
        # Append to the list as a dictionary
        molecule_data.append({
            "Target": target,
            "Molecule_Name": mol_name,
            "SMILES": smiles,
            "Ligand": kind
        })
    
    return molecule_data

In [20]:
df = pd.DataFrame(columns=["Target", "Molecule_Name", "SMILES", "Ligand"])

In [21]:
for kinase in kinase_list:
    target = kinase.lower()
#     print(target)
    actives_file = "all/"+target+"/actives_final.sdf"
    decoys_file = "all/"+target+"/decoys_final.sdf"
    
    actives_data = read_sdf_to_dict(actives_file, target)
    decoys_data = read_sdf_to_dict(decoys_file, target)
    
    combined_ligands = actives_data + decoys_data
    
    df_target = pd.DataFrame(combined_ligands)
    
    df = pd.concat([df, df_target], ignore_index=True)

[18:36:30] Explicit valence for atom # 7 N, 5, is greater than permitted
[18:36:30] ERROR: Could not sanitize molecule ending on line 586033
[18:36:30] ERROR: Explicit valence for atom # 7 N, 5, is greater than permitted
[18:37:14] Explicit valence for atom # 7 N, 5, is greater than permitted
[18:37:14] ERROR: Could not sanitize molecule ending on line 1169873
[18:37:14] ERROR: Explicit valence for atom # 7 N, 5, is greater than permitted


In [22]:
df.to_csv("Kinase_combined_ligand_SMILES.csv")